# FP-growth tree(frequent pattern)

In [237]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
    def inc(self, numOccur):
        self.count += numOccur
    def disp(self, ind = 1):
        print(' ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

In [238]:
rootNode = treeNode('pyramid', 9, None)

In [239]:
rootNode.children['eye'] = treeNode('eye', 13, None)

In [240]:
rootNode.disp()

  pyramid   9
   eye   13


In [241]:
rootNode.children['phoenix'] = treeNode('phoenix', 3, None)

In [242]:
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


# create FP-Growth

In [243]:
def createTree(dataSet, minSup = 1):
    headerTable = {}
    for trans in dataSet: # traverse every line in dataSet
        for item in trans: # traverse every word in line
            # 字典get()函数返回指定键的值，如果值不在字典中返回0。
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans] # because of every list is frozenset dataSet[trans] = 1
            
    print(headerTable)
    
    lessThanMinsup = list(filter(lambda k : headerTable[k] < minSup, headerTable.keys()))
    print(lessThanMinsup)
    for k in lessThanMinsup:
        del(headerTable[k])
        
    print(headerTable)
            
#    for k in list(headerTable):
#       if headerTable[k] < minSup:
#            del(headerTable[k]) # traverse every letter in header table. if in all dataset this letter appeared not larger than minSup delete
#            
#    print(headerTable)
            
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] # 保存计数值及指向每种类型第一个元素的指针
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
#                print("**********")
#                print(localD)
#                print("**********")
        
        if len(localD) > 0:
            orderdItems = [v[0] for v in sorted(localD.items(), key = lambda p : (p[1], p[0]), reverse = True)]
            print(orderdItems)
            updateTree(orderdItems, retTree, headerTable, count) # populate tree with ordered freq itemset 
        print(localD)
        print("\n")
            
    
    return retTree, headerTable

In [244]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = treeNode(items[0], count, inTree) # all count is 1
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]] # 若原来不存在该类别，更新头指针列表
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [245]:
def updateHeader(nodeToTest, targetNode):
    while(nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [246]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [247]:
def loadSimpDat():
    simpDat = [
        ['r', 'z', 'h', 'j', 'p'],
        ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
        ['z'],
        ['r', 'x', 'n', 'o', 's'],
        ['y', 'r', 'x', 'z', 'q', 't', 'p'],
        ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']
    ]
    return simpDat

In [248]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [249]:
simpDat = loadSimpDat()

In [250]:
initSet = createInitSet(simpDat)

In [251]:
myFPtree, myHeaderTab = createTree(initSet, 3)

{'p': 2, 'j': 1, 'r': 3, 'h': 1, 'z': 5, 's': 3, 'w': 1, 'v': 1, 'u': 1, 'x': 4, 't': 3, 'y': 3, 'o': 1, 'n': 1, 'q': 2, 'm': 1, 'e': 1}
['p', 'j', 'h', 'w', 'v', 'u', 'o', 'n', 'q', 'm', 'e']
{'r': 3, 'z': 5, 's': 3, 'x': 4, 't': 3, 'y': 3}
['z', 'r']
{'r': 3, 'z': 5}


['z', 'x', 'y', 't', 's']
{'s': 3, 'x': 4, 't': 3, 'y': 3, 'z': 5}


['z']
{'z': 5}


['x', 's', 'r']
{'s': 3, 'x': 4, 'r': 3}


['z', 'x', 'y', 't', 'r']
{'x': 4, 't': 3, 'y': 3, 'r': 3, 'z': 5}


['z', 'x', 'y', 't', 's']
{'s': 3, 'x': 4, 't': 3, 'y': 3, 'z': 5}




In [252]:
myFPtree.disp()

  Null Set   1
   z   5
    r   1
    x   3
     y   3
      t   3
       s   2
       r   1
   x   1
    s   1
     r   1


# find all paths ending with a given item

In [253]:
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [254]:
def findPrefixPath(basePat, treeNode):
    condPats = {} # create a dictionary to save
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: # if traverse to get prefix path save the path into condPats
            condPats[frozenset(prefixPath[1: ])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [255]:
#test
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [256]:
findPrefixPath('z', myHeaderTab['z'][1]) # only when prefix path lager than 1, it can get a count of current child node

{}

In [259]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    """
    inTree: has benn created tree
    headerTable: head point table
    minSup: the minimum support level
    preFix: set([]) a blank set which include a blank list
    freqItemList: a blank list
    """
    bigL = [v[0] for v in sorted(headerTable.items(), key = lambda p : str(p[1]))]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat) # add element of {t, s, ...} only sort letter v[0]
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        myCondTree, myHead = createTree(condPattBases, minSup)
        if myHead != None:
            print('conditional tree for: ', newFreqSet)
            myCondTree.disp(1)
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [260]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

{'x': 3, 't': 2, 'y': 2, 'z': 2}
['t', 'y', 'z']
{'x': 3}
['x']
{'x': 3}


['x']
{'x': 3}


conditional tree for:  {'s'}
  Null Set   1
   x   3
{}
[]
{}
{'z': 2, 's': 1, 'x': 2, 't': 1, 'y': 1}
['z', 's', 'x', 't', 'y']
{}
{'x': 3, 'z': 3}
[]
{'x': 3, 'z': 3}
['z', 'x']
{'x': 3, 'z': 3}


conditional tree for:  {'y'}
  Null Set   1
   z   3
    x   3
{'z': 3}
[]
{'z': 3}
['z']
{'z': 3}


conditional tree for:  {'x', 'y'}
  Null Set   1
   z   3
{}
[]
{}
{}
[]
{}
{'x': 3, 'y': 3, 'z': 3}
[]
{'x': 3, 'y': 3, 'z': 3}
['z', 'y', 'x']
{'x': 3, 'y': 3, 'z': 3}


conditional tree for:  {'t'}
  Null Set   1
   z   3
    y   3
     x   3
{'y': 3, 'z': 3}
[]
{'y': 3, 'z': 3}
['z', 'y']
{'y': 3, 'z': 3}


conditional tree for:  {'x', 't'}
  Null Set   1
   z   3
    y   3
{'z': 3}
[]
{'z': 3}
['z']
{'z': 3}


conditional tree for:  {'x', 't', 'y'}
  Null Set   1
   z   3
{}
[]
{}
{}
[]
{}
{}
[]
{}
{'z': 3}
[]
{'z': 3}
['z']
{'z': 3}


conditional tree for:  {'t', 'y'}
  Null Set   1
   z   3
{}
